In [1]:
from typing import Annotated # annotated는 타입 힌트를 사용할 때 사용하는 함수
from typing_extensions import TypedDict # TypedDict는 딕셔너리 타입을 정의할 때 사용하는 함수

from langgraph.graph.message import add_messages

# 상태 정의
class State(TypedDict):	
    messages: Annotated[list[str], add_messages]

In [12]:
# 노드 함수 정의
def check_weather_keyword(state: State):
    """사용자 메시지에 '날씨' 키워드가 있는지 확인하는 함수"""
    last_message = state["messages"][-1]
    # '날씨' 키워드를 포함하면 "weather_node"로 이동
    if "날씨" in last_message.content:
        print("날씨 키워드 감지!")
        return "w"
    # 그렇지 않으면 "general_node"로 이동
    else:
        print("일반 키워드 감지!")
        return "g"

def provide_weather(state: State):
    """날씨 정보를 제공하는 노드"""
    print("날씨 노드 실행: 날씨 정보를 제공합니다.")
    return {"messages": ["오늘 날씨는 맑고 따뜻합니다."]}

def provide_general(state: State):
    """일반적인 답변을 제공하는 노드"""
    print("일반 노드 실행: 일반적인 답변을 제공합니다.")
    return {"messages": ["일반적인 답변입니다."]}

In [13]:
from langgraph.graph import StateGraph

# 그래프 구성
graph_builder = StateGraph(State)

graph_builder.add_node("weather_node", provide_weather)
graph_builder.add_node("general_node", provide_general)

In [14]:
from langgraph.graph import START, END

# 엣지 구성
# START에서 조건부 엣지만을 사용하여 분기
graph_builder.add_conditional_edges(
    START,  # <-- 시작 노드를 명시적으로 지정
    check_weather_keyword,
    {
        "w": "weather_node",
        "g": "general_node",
    }
)

In [15]:
# 각 노드가 실행된 후에는 그래프를 종료(END)하도록 엣지를 추가
graph_builder.add_edge("weather_node", END)
graph_builder.add_edge("general_node", END)

In [16]:
graph = graph_builder.compile()

In [17]:
graph.get_graph().print_ascii()

               +-----------+                 
               | __start__ |                 
               +-----------+                 
              ..            ..               
            ..                ..             
          ..                    ..           
+--------------+           +--------------+  
| general_node |           | weather_node |  
+--------------+           +--------------+  
              **            **               
                **        **                 
                  **    **                   
                +---------+                  
                | __end__ |                  
                +---------+                  


In [18]:
print("--- '오늘 날씨 어때?' 입력 ---")
result_weather = graph.invoke({"messages": ["오늘 날씨 어때?"]})
print(f"결과: {result_weather['messages'][-1]}")

--- '오늘 날씨 어때?' 입력 ---
날씨 키워드 감지!
날씨 노드 실행: 날씨 정보를 제공합니다.
결과: content='오늘 날씨는 맑고 따뜻합니다.' additional_kwargs={} response_metadata={} id='4a3b618e-84f3-447d-b13a-2d1069c24d9d'


In [19]:
print("--- '오늘 뭐 먹지?' 입력 ---")
result_general = graph.invoke({"messages": ["오늘 뭐 먹지?"]})
print(f"결과: {result_general['messages'][-1]}")

--- '오늘 뭐 먹지?' 입력 ---
일반 키워드 감지!
일반 노드 실행: 일반적인 답변을 제공합니다.
결과: content='일반적인 답변입니다.' additional_kwargs={} response_metadata={} id='a2f69663-6ef8-43da-bc7d-a06632599f7e'
